In [1]:
import os
import numpy as np
import keras

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras import models
from keras.models import Model, Sequential
from keras import optimizers
from PIL import Image
from keras.applications.vgg16 import preprocess_input
from keras import layers
from keras.layers import Conv2D, GlobalAveragePooling2D, Dense, Flatten
from keras.layers import BatchNormalization, Activation,MaxPooling2D, Lambda, Dropout
from keras.activations import linear
from keras.optimizers import SGD
from keras.datasets import cifar10
from keras.initializers import he_normal

import matplotlib.pylab as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
(train_data,train_labels),(test_data,test_labels) = cifar10.load_data()
train_data = train_data.astype('float32')
test_data = test_data.astype('float32')
train_data = train_data/255
test_data = test_data/255
train_labels = keras.utils.to_categorical(train_labels.astype('float32'))
test_labels = keras.utils.to_categorical(test_labels.astype('float32'))

print(train_data.shape)

170500096/170498071 [==============================] - 2s 0us/step
(50000, 32, 32, 3)


In [0]:
def new_softmax(logits, temperature=1):
   logits = logits/temperature
   return np.exp(logits)/np.sum(np.exp(logits))

In [4]:
from keras.applications import VGG16

conv_base = VGG16(weights = 'imagenet',
                 include_top = False,
                 input_shape = (32, 32, 3))
conv_base.summary()





58892288/58889256 [==============================] - 1s 0us/step






Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
____________________________________________________

In [5]:
model = models.Sequential()
model.add(conv_base)

model.add(layers.Flatten())
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.25))
model.add(layers.Dense(256, activation='relu', kernel_initializer=he_normal()))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))
model.add(Dense(10, name='logits'))
model.add(Activation('softmax'))

model.summary()


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 1, 1, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 512)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)               2048      
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
batch_normalization_2 (Batch (None, 256)               1024      
___

In [0]:
conv_base.trainable = True
set_trainable = False
for layer in conv_base.layers:
  if layer.name == 'block5_conv1':
    set_trainable = True
  if set_trainable:
    layer.trainable = True
  else:
    layer.trainable = False   

In [0]:
from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint, ReduceLROnPlateau

es_cb = EarlyStopping(monitor='val_loss', 
                              patience=10, 
                              verbose=1, 
                              mode='auto')

mc_cb = ModelCheckpoint("model_ck.h5", 
                        monitor='val_loss', 
                        verbose=0,
                        save_best_only=False, 
                        save_weights_only=False, 
                        mode='auto', 
                        period=1)

lr_cb = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,
    patience=5,
    verbose=1
)

In [9]:
from keras.optimizers import RMSprop

model.compile(optimizer=RMSprop(lr=1e-5, decay=1e-6),
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [10]:
history = model.fit(train_data,train_labels,
                    validation_data = (test_data, test_labels),
                    callbacks=[es_cb, mc_cb, lr_cb],
                    epochs=10)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 72s 1ms/step - loss: 2.2619 - acc: 0.3441 - val_loss: 1.2632 - val_acc: 0.5732
Epoch 2/10
50000/50000 [==============================] - 67s 1ms/step - loss: 1.5828 - acc: 0.5168 - val_loss: 1.0948 - val_acc: 0.6353
Epoch 3/10
50000/50000 [==============================] - 66s 1ms/step - loss: 1.3699 - acc: 0.5704 - val_loss: 0.9903 - val_acc: 0.6675
Epoch 4/10
50000/50000 [==============================] - 69s 1ms/step - loss: 1.2349 - acc: 0.6093 - val_loss: 0.9455 - val_acc: 0.6855
Epoch 5/10
50000/50000 [==============================] - 67s 1ms/step - loss: 1.1507 - acc: 0.6351 - val_loss: 0.9030 - val_acc: 0.7031
Epoch 6/10
50000/50000 [==============================] - 66s 1ms/step - loss: 1.0724 - acc: 0.6556 - val_loss: 0.8659 - val_acc: 0.7056
Epoch 7/10
50000/50000 [========

In [0]:
# 精度の履歴を可視化する。
ha = history.history['acc']
va = history.history['val_acc']
epochs = range(len(ha))
plt.plot(epochs, ha, 'ro', label='T acc')
plt.plot(epochs, va, 'g', label='V acc')
plt.legend();

In [0]:
# 損失関数の履歴を可視化する。
ha = history.history['loss']
va = history.history['val_loss']
epochs = range(len(ha))
plt.plot(epochs, ha, 'ro', label='T loss')
plt.plot(epochs, va, 'g', label='V loss')
plt.legend();

In [0]:
(loss,accuracy) = model.evaluate(test_data, test_labels)
print(loss, accuracy)

In [0]:
model_sans_softmax = Model(inputs=model.input, outputs = model.get_layer('logits').output)
new_logits = model_sans_softmax.predict(train_data)
# unsoftened_prob = new_softmax(new_logits, 1)
# print("Unsoftened probabilities " + str(unsoftened_prob[0]))

In [0]:
temperature = 4
softened_prob = new_softmax(new_logits, temperature)
print("Softened probabilities " + str(softened_prob[0]))

In [0]:
#student
model1 = Sequential()
model1.add(Conv2D(32, (3,3), activation='relu', kernel_initializer='he_normal', padding='same', input_shape=(32,32,3)))
model1.add(Conv2D(32, (3,3), activation='relu', kernel_initializer='he_normal', padding='same'))
model1.add(MaxPooling2D((2,2)))
model1.add(Dropout(0.2))
model1.add(Conv2D(64, (3,3), activation='relu', kernel_initializer='he_normal', padding='same'))
model1.add(Conv2D(64, (3,3), activation='relu', kernel_initializer='he_normal', padding='same'))
model1.add(MaxPooling2D((2,2)))
model1.add(Dropout(0.2))
model1.add(Conv2D(128, (3,3), activation='relu', kernel_initializer='he_normal', padding='same'))
model1.add(Conv2D(128, (3,3), activation='relu', kernel_initializer='he_normal', padding='same'))
model1.add(MaxPooling2D((2,2)))
model1.add(Dropout(0.2))
model1.add(Flatten())
model1.add(Dense(128, activation='relu', kernel_initializer='he_normal'))
model1.add(Dropout(0.2))
model1.add(Dense(10, name='logits'))
model1.add(Activation('softmax'))
model1.summary()

In [0]:
logits = model1.get_layer('logits').output
logits = Lambda(lambda x:x/temperature)(logits)
out = Activation('softmax', name='soft')(logits)
new_student = Model(inputs=model1.input, outputs=out)
new_student.summary()

In [0]:
new_student.compile(optimizer=RMSprop(lr=1e-5, decay=1e-6),
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [0]:
history2 = new_student.fit(train_data,softened_prob,
                           validation_data = (test_data, test_labels),
                           callbacks=[es_cb, mc_cb, lr_cb],
                           epochs=10)

In [0]:
(loss,accuracy) = new_student.evaluate(test_data,test_labels)
print(loss, accuracy)